In [2]:
import tensorflow as tf
from tensorflow.keras import optimizers, applications
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import ceil

import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [3]:
model_filename = '/Users/nurajymnaukanova/Desktop/dataset thyroid/model.h5'

# saved filename of model weights
weights_filename = '/Users/nurajymnaukanova/Desktop/modelweights.h5'
modelcheckpoint_filename = '/Users/nurajymnaukanova/Desktop/modelcheckpoint.h5'

# saved filename of result csv
results_filename = '/Users/nurajymnaukanova/Desktop/resultultra.csv'

# directories of dataset
train_data_dir = '/Users/nurajymnaukanova/Desktop/dataset thyroid/train'
# validation_data_dir = '/content/drive/My Drive/FYP/Project/data/validation'
test_data_dir = '/Users/nurajymnaukanova/Desktop/dataset thyroid/test'
# output_dir = '/content/drive/My Drive/FYP/Project/output'

# dimensions of the architecture
img_width, img_height = 224, 224

# number of channels for the architecture
channels = 3

# number of epochs
epochs = 500

# batch sizes
training_batch_size = 122
validation_batch_size = 122

# learning rate
learning_rate = 0.0001

# class mode
class_mode = 'categorical'

# total number of classes
total_classes = 3

In [4]:
def summary():
	# model summary
	vgg16.summary()

	# graphing our training and validation
	accuracy = history.history['accuracy']
	val_accuracy = history.history['val_accuracy']
	loss = history.history['loss']
	val_loss = history.history['val_loss']
	epochs = range(len(accuracy))
	plt.plot(epochs, accuracy, 'r', label = 'Training accuracy')
	plt.plot(epochs, val_accuracy, 'b', label = 'Validation accuracy')
	plt.title('Training and Validation Accuracy')
	plt.ylabel('Accuracy')
	plt.xlabel('Epoch')
	plt.legend()
	plt.figure()
	plt.plot(epochs, loss, 'r', label = 'Training Loss')
	plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
	plt.title('Training and Validation Loss')
	plt.ylabel('Loss')
	plt.xlabel('Epoch')
	plt.legend()
	plt.show()

def saveModel():
	model.save(model_filename)
	model.save_weights(weights_filename)

	model_json = model.to_json()
	with open("model_in_json.json", "w") as json_file:
		json.dump(model_json, json_file)

In [5]:
model = applications.VGG16(include_top = False, weights = 'imagenet', classes = total_classes, input_shape = (img_width, img_height, channels))
flat = Flatten()(model.layers[-1].output)
output = Dense(total_classes, activation = "softmax")(flat)
vgg16 = Model(inputs = model.inputs, outputs = output)

train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                    rotation_range = 90,
                                    brightness_range = [0.2, 1.0],
                                    shear_range = 0.2,
                                    zoom_range = 0.3,
                                    width_shift_range = 1.0,
                                    height_shift_range = 1.0,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    validation_split = 0.4)

# validation_datagen = ImageDataGenerator(rescale = 1. / 255)

# current time
start = datetime.datetime.now()

In [6]:
# training data
training_set = train_datagen.flow_from_directory(
                                		train_data_dir,
                                		target_size = (img_width, img_height),
                                		batch_size = training_batch_size,
                                		class_mode = class_mode,
										#save_to_dir = output_dir,
										#save_prefix = 'Aug',
										subset = "training",
										seed = 42,
                                		shuffle = True)

nb_train_samples = len(training_set.filenames)

Found 1160 images belonging to 3 classes.


In [7]:
# validation data
validation_set = train_datagen.flow_from_directory(
                                  		train_data_dir,
                                  		target_size = (img_width, img_height),
                                  		batch_size = validation_batch_size,
                                  		class_mode = class_mode,
										subset = "validation",
										seed = 42,
                                  		shuffle = True)

nb_validation_samples = len(validation_set.filenames)

Found 770 images belonging to 3 classes.


In [8]:
vgg16.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),  # Use learning_rate instead of lr
              metrics=['accuracy'])

In [9]:
# EarlyStopping callback to stop training if the validation loss does not improve
es = EarlyStopping(monitor='val_loss',
                   min_delta=1e-5,
                   patience=5,
                   verbose=1,
                   mode='min',
                   restore_best_weights=True)

# ModelCheckpoint callback to save the best model weights based on validation loss
# ModelCheckpoint callback to save the best model weights based on validation loss
# Ensure the filepath ends with .weights.h5 when saving only weights
modelcheckpoint_filename = '/Users/nurajymnaukanova/Desktop/modelcheckpoint.weights.h5'  # Updated filename

mc = ModelCheckpoint(modelcheckpoint_filename,
                     monitor='val_loss',
                     mode='min',
                     save_weights_only=True,  # Only save the weights, not the full model
                     save_best_only=True)


# Use the fit method instead of fit_generator (TensorFlow 2.x)
history = vgg16.fit(
    training_set,
    steps_per_epoch=nb_train_samples // training_batch_size,
    epochs=epochs,
    callbacks=[es, mc],
    validation_data=validation_set,
    validation_steps=nb_validation_samples // validation_batch_size)


Epoch 1/500


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 288s 32s/step - accuracy: 0.3496 - loss: 1.1855 - val_accuracy: 0.4686 - val_loss: 0.9425
Epoch 2/500
1/9 ━━━━━━━━━━━━━━━━━━━━ 3:55 29s/step - accuracy: 0.5656 - loss: 0.9035

2025-05-10 06:00:48.316865: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 317ms/step - accuracy: 0.5656 - loss: 0.9035 - val_accuracy: 0.4474 - val_loss: 0.8174
Epoch 3/500


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 292s 32s/step - accuracy: 0.5187 - loss: 0.9283 - val_accuracy: 0.4713 - val_loss: 0.9352
Epoch 4/500
1/9 ━━━━━━━━━━━━━━━━━━━━ 3:44 28s/step - accuracy: 0.5410 - loss: 0.8717

2025-05-10 06:06:10.550719: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


9/9 ━━━━━━━━━━━━━━━━━━━━ 30s 290ms/step - accuracy: 0.5410 - loss: 0.8717 - val_accuracy: 0.5789 - val_loss: 0.8312
Epoch 5/500


2025-05-10 06:06:12.834184: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


KeyboardInterrupt: 

In [12]:
summary()  # This will plot the accuracy and loss graphs
saveModel()  # This will save the model and its weights

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │        75,267 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,369,867 (169.26 MB)

 Trainable params: 14,789,955 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 29,579,912 (112.84 MB)

NameError: name 'history' is not defined

In [15]:
# Evaluate the model on the validation data
eval_loss, eval_accuracy = vgg16.evaluate(
    validation_set,  # Validation data generator
    steps=nb_validation_samples // validation_batch_size,  # Number of validation steps
    verbose=1  # To display progress
)

print(f"[INFO] Validation Loss: {eval_loss:.4f}")
print(f"[INFO] Validation Accuracy: {eval_accuracy*100:.2f}%")
end = datetime.datetime.now()
elapsed = end - start
print('[INFO] Time Elapsed: ', elapsed)


6/6 ━━━━━━━━━━━━━━━━━━━━ 45s 7s/step - accuracy: 0.4709 - loss: 0.9473
[INFO] Validation Loss: 0.9390
[INFO] Validation Accuracy: 47.95%
[INFO] Time Elapsed:  0:15:35.564587


In [16]:
print("[INFO] Accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end = datetime.datetime.now()
elapsed = end - start
print('[INFO] Time Elapsed: ', elapsed)

[INFO] Accuracy: 47.95%
[INFO] Loss: 0.9390110373497009
[INFO] Time Elapsed:  0:16:18.372289
